### CNN Image Classification Problem 

Identifying Refugee Camps in the Middle East

Here the objective is to train a CNN on satellite images to identify refugee camps in the Middle East.  Here we are dealing with a real world problem and a limited amount of data.  We'll use a data augmentation feature to suplement our existing dataset.  

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt
import os, sys
import itertools, functools
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import scale, LabelBinarizer
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Random seed for numpy
np.random.seed(18937)
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam

import os
import argparse
from shutil import copy
from pathlib import Path
from multiprocessing import Pool, TimeoutError

Using TensorFlow backend.


In [66]:
# functions to create folders and copy files 
import os


def create_fold_dirs(output_path, num_folds):
    """
    Create train and test directories in output path
    :param output_path: Path to the output
    :param num_folds: the number of folds
    """
    for i in range(num_folds):
        for word in ["train", "test"]:
            os.mkdir(os.path.join(str(output_path), word + "_fold" + chr(65+i)))
    

def copy_file(file, source, output):
    f = file
    full_path = os.path.join(source, f)
    full_output= os.path.join(output, f)
    folder = full_output[:full_output.rfind("/")]
    if not Path(folder).exists():
        Path(folder).mkdir()
    if not Path(full_output).exists():
        print("Copying...{}".format(f))
        copy(full_path, full_output)    
           

In [77]:
img_dir = '/dsa/home/jamgtn/experimental/data/refugee_camp_data/Noncamp'
out_path2 = '/dsa/home/jamgtn/experimental/data/refugee_camp_data/n_test/noncamp'
i = 0
for filename in os.listdir(img_dir):
    if filename.endswith("10.png"): 
        copy_file(filename, img_dir, out_path2)
        #print(os.path.join(img_dir, filename))
        continue
    else:
        continue  

#############################
#    i += 1
#    if i > 2050:
#        break  # otherwise the generator would loop indefinitely

Copying...noncamp,altinozu,36.35925,36.17987-chip10.png
Copying...noncamp,altinozu,36.35925,36.18570-chip10.png
Copying...noncamp,altinozu,36.37371,36.19153-chip10.png
Copying...noncamp,altinozu,36.37371,36.18570-chip10.png
Copying...noncamp,altinozu,36.37371,36.17987-chip10.png
Copying...noncamp,altinozu,36.36648,36.19153-chip10.png
Copying...noncamp,altinozu,36.36648,36.18570-chip10.png
Copying...noncamp,altinozu,36.36648,36.17987-chip10.png
Copying...noncamp,altinozu,36.35925,36.19153-chip10.png
Copying...noncamp,apaydin,36.35856,36.23933-chip10.png
Copying...noncamp,apaydin,36.35855,36.23349-chip10.png
Copying...noncamp,apaydin,36.35855,36.22766-chip10.png
Copying...noncamp,apaydin,36.35132,36.23933-chip10.png
Copying...noncamp,apaydin,36.35132,36.23349-chip10.png
Copying...noncamp,apaydin,36.35132,36.22766-chip10.png
Copying...noncamp,apaydin,36.34409,36.23933-chip10.png
Copying...noncamp,apaydin,36.34409,36.23349-chip10.png
Copying...noncamp,apaydin,36.34409,36.22766-chip10.png
C

## Data Augmentation 

In [59]:
# this cell is not used in the model, just here to demonstrate how datagen works.  

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.0,
        zoom_range=0.0,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('/dsa/home/jamgtn/experimental/data/refugee_camp_data/n_test/noncamp/noncamp,altinozu,36.35925,36.17987-chip10.png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x)

print(x.shape)

train = '/dsa/home/jamgtn/experimental/data/refugee_camp_data/train'

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=train, save_prefix='camps', save_format='png'):
    i += 1
    if i > 50:
        break  # otherwise the generator would loop indefinitely

[[[[  71.   76.   68.]
   [  68.   83.   75.]
   [  84.   90.   77.]
   ..., 
   [ 181.  201.  204.]
   [ 168.  182.  184.]
   [ 152.  167.  168.]]

  [[  57.   67.   58.]
   [  77.   84.   74.]
   [ 103.  109.   98.]
   ..., 
   [ 187.  204.  211.]
   [ 187.  204.  211.]
   [ 195.  206.  211.]]

  [[  71.   76.   68.]
   [  87.   93.   83.]
   [ 109.  117.  106.]
   ..., 
   [ 187.  204.  211.]
   [ 207.  221.  227.]
   [ 228.  237.  242.]]

  ..., 
  [[ 119.  140.  147.]
   [ 107.  131.  136.]
   [ 119.  140.  147.]
   ..., 
   [ 102.  124.  131.]
   [  89.  109.  115.]
   [ 107.  131.  136.]]

  [[ 119.  140.  147.]
   [ 102.  124.  131.]
   [ 119.  140.  147.]
   ..., 
   [ 102.  120.  121.]
   [ 107.  131.  136.]
   [ 135.  151.  152.]]

  [[ 140.  163.  167.]
   [ 137.  156.  163.]
   [ 123.  151.  163.]
   ..., 
   [ 102.  124.  131.]
   [ 119.  140.  147.]
   [ 140.  163.  167.]]]]
(1, 256, 256, 3)


In [4]:
# let's take a peak at the files in the directory
import os
indir = '/dsa/home/jamgtn/experimental/data/refugee_camp_data/n_test/noncamp'
for root, dirs, filenames in os.walk(indir):
    for f in filenames:
        print(f)

#for root, dirs, filenames in load_img(indir):
#    for f in filenames:
#        print(f)
        

noncamp,islahiye,36.61867,36.97753-chip10.png
noncamp,kahmanaras_turkoglu,36.96352,37.56458-chip10.png
noncamp,kahmanaras_turkoglu,36.97088,37.57041-chip10.png
noncamp,kahmanaras_turkoglu,36.97823,37.55875-chip10.png
noncamp,karkamis,38.03530,36.87465-chip10.png
noncamp,malatya_beydagi,38.12888,38.35624-chip10.png
noncamp,malatya_beydagi,38.17350,38.33291-chip10.png
noncamp,altinozu,36.35925,36.18570-chip10.png
noncamp,apaydin,36.34409,36.22766-chip10.png
noncamp,king_abdullah_park,36.02617,32.52403-chip10.png
noncamp,malatya_beydagi,38.17351,38.35624-chip10.png
noncamp,altinozu,36.35925,36.17987-chip10.png
noncamp,altinozu,36.37371,36.19153-chip10.png
noncamp,malatya_beydagi,38.15863,38.34457-chip10.png
noncamp,malatya_beydagi,38.12888,38.35040-chip10.png
noncamp,altinozu,36.36648,36.17987-chip10.png
noncamp,altinozu,36.37371,36.18570-chip10.png
noncamp,apaydin,36.35132,36.23933-chip10.png
noncamp,islahiye,36.61137,36.98337-chip10.png
noncamp,malatya_beydagi,38.13632,38.33291-chip10.p

## Build Model 


In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

z = load_img('/dsa/home/jamgtn/experimental/data/refugee_camp_data/n_test/noncamp/noncamp,altinozu,36.35925,36.17987-chip10.png')  # this is a PIL image
x = img_to_array(z)  # this is a Numpy array with shape (3, 150, 150)
print(x.shape)

n_col = x.shape
print(n_col)

xt = np.transpose(x)
inputs = Input(shape=(784,))

n_col

(150, 150, 3)
(150, 150, 3)


(150, 150, 3)

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(n_col))) # n_col is defined by the shape above 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [4]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/dsa/home/jamgtn/experimental/data/refugee_camp_data/n_train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/dsa/home/jamgtn/experimental/data/refugee_camp_data/n_test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')


Found 108 images belonging to 2 classes.
Found 51 images belonging to 2 classes.


In [6]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2881 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=1184 // batch_size)
model.save_weights('first_try.h5')  #  save weights after training or during training

Epoch 1/50
180/180 [==============================] - 67s - loss: 0.2254 - acc: 0.9206 - val_loss: 0.2131 - val_acc: 0.9411
Epoch 2/50
180/180 [==============================] - 66s - loss: 0.0495 - acc: 0.9834 - val_loss: 0.1657 - val_acc: 0.9600
Epoch 3/50
180/180 [==============================] - 67s - loss: 0.0431 - acc: 0.9860 - val_loss: 0.2554 - val_acc: 0.9411
Epoch 4/50
180/180 [==============================] - 66s - loss: 0.0079 - acc: 0.9971 - val_loss: 0.3332 - val_acc: 0.9400
Epoch 5/50
180/180 [==============================] - 66s - loss: 0.0147 - acc: 0.9970 - val_loss: 0.4245 - val_acc: 0.9421
Epoch 6/50
180/180 [==============================] - 67s - loss: 0.0134 - acc: 0.9958 - val_loss: 0.2931 - val_acc: 0.9400
Epoch 7/50
180/180 [==============================] - 66s - loss: 0.0025 - acc: 0.9993 - val_loss: 0.3676 - val_acc: 0.9611
Epoch 8/50
180/180 [==============================] - 67s - loss: 9.9919e-04 - acc: 0.9997 - val_loss: 0.4267 - val_acc: 0.9611
Epoc